In [36]:
import pathlib

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import random as rn
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [37]:
data_dir="work/keras/carclassfication_model/data/car_data/test"
image_size = (224, 224)
batch_size = 50
inputs = keras.Input(shape=(224, 224, 3))

In [38]:
# train_ds, val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     validation_split=0.2,
#     subset="both",
#     seed=66,  # test across diff seed values
#     labels="inferred",
#     label_mode="categorical",
#     image_size=image_size,
#     batch_size=batch_size,
# )
# train_ds = preprocess_input(train_ds)
# val_ds = preprocess_input(val_ds)
#
# class_names = train_ds.class_names

In [39]:
transform = ImageDataGenerator(preprocessing_function= keras.applications.resnet.preprocess_input, validation_split=0.2)

train_ds = transform.flow_from_directory(
    data_dir,
    subset="training", seed = 42,target_size=image_size,batch_size=batch_size
)
val_ds = transform.flow_from_directory(
    data_dir,
    subset="validation", seed = 42,target_size=image_size,batch_size=batch_size
)

Found 6517 images belonging to 196 classes.
Found 1524 images belonging to 196 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', input_tensor=inputs, include_top=True)
last_layer = base_model.layers[-2].output
out = Dense(units = len(train_ds.class_indices), activation = 'softmax', name = 'ouput')(last_layer)

new_base_model = Model(inputs = inputs, outputs = out)

for layer in new_base_model.layers[:-25]:
  layer.trainable = False

model = Model(inputs = inputs, outputs = out)

model.compile (
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    steps_per_epoch=len(train_ds),
    validation_steps=len(val_ds)
)

Epoch 1/5
131/131 [==============================] - 945s 7s/step - loss: 3.9899 - accuracy: 0.1547 - val_loss: 7.3171 - val_accuracy: 0.1004
Epoch 2/5
 19/131 [===>..........................] - ETA: 16:46 - loss: 1.8569 - accuracy: 0.5179

In [ ]:
# Plot the Loss

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.epoch))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
car_path = []
from tensorflow.keras.utils import img_to_array

class_names = list(train_ds.class_indices.keys())

validation_path = pathlib.Path("data/car_data/test_subset1/")
valid = list(validation_path.glob('*/*'))
rn_car = rn.randint(0, len(valid))
car = valid[rn_car]

print("Actual car {}".format(car))

image = load_img(car, target_size=image_size)
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
temp = plt.imread(car)
# plt.imshow(temp)
# plt.show()
image = keras.applications.resnet50.preprocess_input(image)
predictions = model.predict(image)
score = tf.nn.softmax(predictions[0])

print(
  "Prediction: {} with a {:.2f} percent confidence."
  .format(class_names[np.argmax(score)], 100 * np.max(score)))